In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

from astroML.plotting          import hist
from astropy.io                import fits
from astropy.modeling          import models, fitting
from datetime                  import datetime
from image_registration        import cross_correlation_shifts
from glob                      import glob
from matplotlib.ticker         import MaxNLocator
from matplotlib                import style
from os                        import listdir
from least_asymmetry.asym      import actr, moments, fitgaussian
from multiprocessing           import cpu_count, Pool
from numpy                     import min as npmin, max as npmax, zeros, arange, sum, float, isnan, hstack
from numpy                     import int32 as npint, round as npround, nansum as sum, nanstd as std
from os                        import environ
from pandas                    import DataFrame, read_csv, read_pickle, scatter_matrix
from photutils                 import CircularAperture, CircularAnnulus, aperture_photometry, findstars
from pylab                     import ion, gcf, sort, linspace, indices, median, mean, std, empty, figure, transpose, ceil
from pylab                     import concatenate, pi, sqrt, ones, diag, inf, rcParams, isnan, isfinite, array, nanmax
from pylab                     import figure, plot, imshow
from seaborn                   import *
from scipy.special             import erf
from scipy                     import stats
from sklearn.externals         import joblib
from socket                    import gethostname
from statsmodels.robust        import scale
from statsmodels.nonparametric import kde
from sys                       import exit
from time                      import time, localtime

from numpy                     import zeros, nanmedian as median, nanmean as mean, nan
from sys                       import exit
from sklearn.externals         import joblib

import numpy as np

In [ ]:
startFull = time()

**Master Class for Exoplanet Time Series Observation Photometry**

In [ ]:
from ExoplanetTSO_Auxiliary import wanderer

In [ ]:
rcParams['image.interpolation'] = 'None'
rcParams['image.cmap']          = 'Blues_r'
rcParams['axes.grid']           = False

As an example, Spitzer data is expected to be store in the directory structure:

`$HOME/PLANET_DIRECTORY/PLANETNAME/data/raw/AORDIR/CHANNEL/bcd/`

EXAMPLE:

1. On a Linux machine
2. With user `tempuser`,
3. And all Spitzer data is store in `Research/Planets`
4. The planet named `Happy-5b`
5. Observed during AOR r11235813
6. In CH2 (4.5 microns)

The `loadfitsdir` should read as: `/home/tempuser/Research/Planets/HAPPY5/data/raw/r11235813/ch2/bcd/`

In [ ]:
planetName      = 'example'
planetDirectory = '/path/to/example/base/directory/'

dataDir     = environ['HOME'] + planetDirectory + planetName + '/data/raw/'

# This is only a list as an example because it the following code can be used to combine multiple AORs
AORs    = ['r12345678', 'r11234580']

channel = 'ch2/'

dataSub = 'bcd/'

fileExt = '*bcd.fits'
uncsExt = '*bunc.fits'

In [ ]:
iAOR        = INTEGER
AORNow      = AORs[iAOR]
loadfitsdir = dataDir + AORNow + '/' + channel + dataSub
print(loadfitsdir)

In [ ]:
fitsFilenames = glob(loadfitsdir + fileExt)
uncsFilenames = glob(loadfitsdir + uncsExt)

In [ ]:
len(fitsFilenames), len(uncsFilenames)

In [ ]:
fitsFilenames

In [ ]:
uncsFilenames

# Load ExoplanetTSO Class

Necessary Constants Spitzer
---

In [ ]:
ppm             = 1e6
y,x             = 0,1

yguess, xguess  = 15., 15.   # Specific to Spitzer circa 2010 and beyond
filetype        = 'bcd.fits' # Specific to Spitzer Basic Calibrated Data

Load Stored Instance from Save Files
---

If this is **not** your time, then you can load a saved file in the `loadfitsdir` directory that has the filename:

`PLANETNAME_Wanderer_METHOD_DATE.pickle.save` 

```python
method = 'mean'
example_wanderer_mean = wanderer(loadfitsdir=loadfitsdir, filetype=filetype, 
                                            yguess=yguess, xguess=xguess, method=method)

example_wanderer_mean.load_data_from_save_files(savefiledir='./SaveFiles/', 
                        saveFileNameHeader=planetName+'_Wanderer_Mean_', saveFileType='.pickle.save')
```

```python
method = 'median'
example_wanderer_median = wanderer(loadfitsdir=loadfitsdir_ModA, filetype=filetype, 
                                            yguess=yguess, xguess=xguess, method=method)

example_wanderer_median.load_data_from_save_files(savefiledir='./SaveFiles/', saveFileNameHeader='Example_Wanderer_Median_', saveFileType='.pickle.save')
```

Start a New Instance with Median for the Metric
---

In [ ]:
method = 'median'

print('Initialize an instance of `wanderer` as `example_wanderer_median`\n')
example_wanderer_median = wanderer(fitsFileDir=loadfitsdir, filetype=filetype, telescope='Spitzer',
                                            yguess=yguess, xguess=xguess, method=method, nCores=cpu_count())

In [ ]:
print('Load Data From Fits Files in ' + loadfitsdir + '\n')
example_wanderer_median.load_data_from_fits_files()

In [ ]:
print('Skipping Load Data From Save Files in ' + loadfitsdir + '\n')
# example_wanderer_median.load_data_from_save_files(savefiledir='./SaveFiles/', saveFileNameHeader='Example_Wanderer_Median_', saveFileType='.pickle.save')

In [ ]:
print('Find, flag, and NaN the "Bad Pixels" Outliers' + '\n')
example_wanderer_median.find_bad_pixels()

In [ ]:
print('Fit for All Centers: Flux Weighted, Gaussian Fitting, Gaussian Moments, Least Asymmetry' + '\n')
# example_wanderer_median.fit_gaussian_centering()
example_wanderer_median.fit_flux_weighted_centering()
# example_wanderer_median.fit_least_asymmetry_centering()
# example_wanderer_median.fit_all_centering() # calling this calls least_asymmetry, which does not work :(

In [ ]:
start = time()
example_wanderer_median.mp_lmfit_gaussian_centering()
print('Operation took {} seconds with {} cores'.format(time()-start, example_wanderer_median.nCores))

# start = time()
# example_wanderer_median.mp_fit_flux_weighted_centering()
# print('Operation took {} seconds with {} cores'.format(time()-start, example_wanderer_median.nCores))

In [ ]:
example_wanderer_median.centering_FluxWeight[0]

In [ ]:
print('Measure Background Estimates with All Methods: Circle Masked, Annular Masked, KDE Mode, Median Masked' + '\n')
# example_wanderer_median.measure_background_circle_masked()
# example_wanderer_median.measure_background_annular_mask()
# example_wanderer_median.measure_background_KDE_Mode()
# example_wanderer_median.measure_background_median_masked()
example_wanderer_median.measure_all_background()

In [ ]:
example_wanderer_median.measure_effective_width()

In [ ]:
example_wanderer_median.flux_TSO_df = DataFrame()

In [ ]:
print('Iterating over Background Techniques, Centering Techniques, Aperture Radii' + '\n')
centering_choices  = ['Gaussian_Fit']#, 'Gaussian_Mom', 'FluxWeighted']#, 'LeastAsymmetry']
background_choices = ['AnnularMask']#example_wanderer_median.background_df.columns
staticRads         = [2.5 ]# aperRads = np.arange(1, 6,0.5)
varRads            = [None]# aperRads = np.arange(1, 6,0.5)

# example_wanderer_median.compute_flux_over_time_over_aperRad(staticRads, centering_choices, \
#                                                             background_choices, useTheForce=True)
startMPFlux = time()
example_wanderer_median.mp_compute_flux_over_time(staticRads[0], centering_choices[0], \
                                                            background_choices[0], useTheForce=True)
print('Entire Pipeline took {} seconds'.format(time() - startMPFlux))

In [ ]:
for key in flux_TSO_df_mp_save.keys():
    plot(example_wanderer_median.flux_TSO_df[key])
plot(flux_TSO_df_mp_save[key])
print(sum(abs(example_wanderer_median.flux_TSO_df[key] - flux_TSO_df_mp_save[key])))

In [ ]:
print('Saving `example_wanderer_median` to a set of pickles for various Image Cubes and the Storage Dictionary')

savefiledir         = environ['HOME']+'/Research/Planets/'+planetName+'/SaveFiles/'
saveFileNameHeader  = planetName+'_'+ AORNow +'_Median'
saveFileType        = '.pickle.save'

print(savefiledir)
print(saveFileNameHeader)
print(saveFileType)
example_wanderer_median.save_data_to_save_files(savefiledir=savefiledir, \
                                                saveFileNameHeader=saveFileNameHeader, \
                                                saveFileType=saveFileType)

In [ ]:
print('Entire Pipeline took {} seconds'.format(time() - startFull))

# Analysis

In [ ]:
print('Saving `example_wanderer_median` to a set of pickles for various Image Cubes and the Storage Dictionary')

example_wanderer_median_dict = {}
for AORNow in AORs:
    savefiledir         = environ['HOME']+'/Research/Planets/'+planetName+'/SaveFiles/'
    saveFileNameHeader  = planetName+'_'+ AORNow +'_Median'
    saveFileType        = '.pickle.save'

    print(savefiledir)
    print(saveFileNameHeader)
    print(saveFileType)

    method = 'median'

    print('Initialize an instance of `wanderer` as `example_wanderer_median`\n')
    example_wanderer_median_dict[AORNow] = wanderer(fitsFileDir=loadfitsdir, filetype=filetype, telescope='Spitzer',
                                                yguess=yguess, xguess=xguess, method=method, nCores=cpu_count())

    example_wanderer_median_dict[AORNow].load_data_from_save_files(savefiledir=savefiledir, 
                                                                  saveFileNameHeader=saveFileNameHeader, 
                                                                  saveFileType=saveFileType)

In [ ]:
fig=figure(figsize=(20,10),dpi=100)
ax = fig.add_subplot(111)
for aorkey in list(example_wanderer_median_dict.keys())[::-1]:
    ex_dict = example_wanderer_median_dict[aorkey]
    ax.plot(ex_dict.timeCube, ex_dict.flux_TSO_df[key], '.', label=aorkey)

ax.legend(loc=0, fontsize=20)
ax.set_ylim(ymin, ymax)
ax.set_xlim(xmin, xmaxqatar2)

# Things to Work on